# MNIST using Keras CNN Ensemble(0.996 accuracy)
This notebook covers CNN, ensemble and visualization process step-by-step.
I hope you find this notebook useful!

* [1. Import required libraries](#1)
* [2. Data preparation](#2)
    * [2.1 load dataset](#2.1)
    * [2.2 Optimize data](#2.2)
    * [2.3 Data Augmentation](#2.3)
* [3. Build CNN](#3)
    * [3.1 Define the model](#3.1)
* [4. Training](#4)
* [5. Evaluate the model](#5)
    * [5.1 Predictions correlation matrix](#5.1)
    * [5.2 Training and validation curves](#5.2)
* [6. Create submission](#6)

<a id="1"></a>
<h1 style='background:navy; border:0; color:white'><center>1. Import required libraries</center></h1>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

<a id="2"></a>
<h1 style='background:navy; border:0; color:white'><center>2. Data preparation</center></h1>

<a id="2.1"></a>
## 2.1 load dataset

In [ ]:
train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

<a id="2.2"></a>
## 2.2 Optimize data

In [ ]:
# divide training data into features and labels
X_train = train.iloc[:,1:]
y_train = train.iloc[:,0]

In [ ]:
# Normalize and reshape image
X_train = X_train.values.reshape(-1, 28, 28, 1)/255.
test = test.values.reshape(-1, 28, 28, 1)/255.
# One Hot encoding the label
y_train = to_categorical(y_train, 10)

<a id="2.3"></a>
## 2.3 Data Augmentation

In [ ]:
datagen = ImageDataGenerator(
            rotation_range=15,
            width_shift_range=0.1,
            height_shift_range=0.1,
            zoom_range=0.1,
            shear_range=0.2
            )

<a id="3"></a>
<h1 style='background:navy; border:0; color:white'><center>3. Build CNN</center></h1>

<a id="3.1"></a>
## 3.1 Define the model

In [ ]:
def create_model():
    model = Sequential()

    model.add(Conv2D(32, (3,3), padding='same', input_shape=X_train.shape[1:], activation='relu'))
    model.add(Conv2D(32, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(2,2))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(2,2))
    model.add(Dropout(0.2))

    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(2,2))
    model.add(Dropout(0.2))
    
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

<a id="4"></a>
<h1 style='background:navy; border:0; color:white'><center>4. Training</center></h1>

In [ ]:
EPOCHS = 30
BATCH_SIZE = 50
ENSEMBLES = 5 # number of models to ensemble
result_list = [] # store results for correlation matrix
histories = [] # store histories for training and validation curves
results = np.zeros((test.shape[0],10))

callback_list = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.25, patience=2),
    EarlyStopping(monitor='val_loss', min_delta=0.0005, patience=4)
]

for i in range(ENSEMBLES):
    # split training and validation sets
    X_train_tmp, X_val, y_train_tmp, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=i)
    # create model
    model = create_model()
    # fit the model
    print('training No.', i)
    history = model.fit(datagen.flow(X_train_tmp, y_train_tmp, batch_size=BATCH_SIZE),
                   verbose=0,
                   epochs=EPOCHS,
                   callbacks=callback_list,
                   validation_data=(X_val, y_val),
                   steps_per_epoch=X_train_tmp.shape[0] // BATCH_SIZE)
    # save results
    histories.append(history)
    result = model.predict(test)
    results += result
    result_list.append(result)

<a id="5"></a>
<h1 style='background:navy; border:0; color:white'><center>5. Evaluate the model</center></h1>

<a id="5.1"></a>
## 5.1 Predictions correlation matrix

In [ ]:
# check correlation of each predictions
corr_preds = pd.DataFrame([np.argmax(result, axis=1) for result in result_list]).T.corr()
fig = sns.heatmap(corr_preds, annot=True, fmt='.3f', cmap='rainbow')
fig.set_title('Predictions correlation matrix', fontsize=16, y=1.05)
plt.show()

<a id="5.2"></a>
## 5.2 Training and validation curves

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(12,6))

for e in range(ENSEMBLES):
    loss = histories[e].history['loss']
    val_loss = histories[e].history['val_loss']
    acc = histories[e].history['accuracy']
    val_acc = histories[e].history['val_accuracy']
    ax[0].set_title('loss')
    ax[0].plot(loss, 'b', linewidth=1)
    ax[0].plot(val_loss, 'r', linewidth=1)
    ax[0].grid(color='black', linestyle='-', linewidth=0.2)
    ax[1].set_title('accuracy')
    ax[1].plot(acc, 'b', linewidth=1)
    ax[1].plot(val_acc, 'r', linewidth=1)
    ax[1].grid(color='black', linestyle='-', linewidth=0.2)
    
ax[0].legend(['Training loss', 'Validation loss'], shadow=True)     
ax[1].legend(['Training accuracy', 'Validation accuracy'], shadow=True)

plt.tight_layout()
plt.show()

<a id="6"></a>
<h1 style='background:navy; border:0; color:white'><center>6. Create submission</center></h1>

In [ ]:
results = np.argmax(results, axis=1)
results = pd.Series(results, name='Label')
submission = pd.concat([pd.Series(range(1,28001), name='ImageID'), results], axis=1)
submission.to_csv('submission.csv', index=False)